# fMRIPrep pre-processing and post-processing - connectivity matrices extraction using a parcellation

This notebook is intented to give a brief overview of a pipeline to:

1) Download data from Beluga, an HPC  
2) Verify the data with bids-validator using a Docker container  
3) Run the pre-processing with fMRIPrep using a Docker container  
4) Create and run a post-processing pipeline in Nilearn to obtain connectivity matrices

## Background

The data used in this tutorial is anonymized data from the Prevent-AD Cohort. Data from the Prevent-AD is open and available at this [address](https://portal.conp.ca/dataset?id=projects/preventad-open). Unfortunately, the images from the cohort on the CONP are only available in .mnc format. While .mnc images are tranformable to .nii format, they are not transformable in BIDS at this time.

As such, the tutorial used closed data from the Prevent-AD cohort which was readily available in BIDS format. As the data is not open, the participant ID was anonymized and the data is not available to be reproduced.

However, the tools used and the tutorial should be applicable to any bids-validated datasets, which is why we run the bids-validator first. 

## 1) Downloading data from Beluga

In this step, we simply download the data from Beluga. The exact code is not shared as it would leak the actual participant ID. However, I share below the command that would be used to download the data. 

```scp -r user@beluga.computecanada.ca:/path/to/directory/to/copy /directory/local/computer/to/copy```

In this ```scp``` command, the ```-r``` argument serves to copy whole directories to a local computer. It is important to put ```:``` after your user name for the HPC. The path after the ```:``` refers to the path on the supercomputer that needs to be copied to the local computer. After this path, a white space and a path refering to the folder on the local computer where the folder from the HPC should be copied.

Should you want to copy a folder from your computer to the HPC (e.g. after the pre-processing), you simply need to invert the order of the paths.

```scp -r /directory/local/computer/to/copy/to/HPC user@beluga.computecanada.ca:/path/to/remote/directory/```

## 2) Verify BIDS validity with bids-validator

Once the data is downloaded, and in our case, anonymized, we are ready to validate our bids! 

While this might seem redundant when running the bids-validator on our local computer (since fMRIPrep also runs the bids-validator before launching the pre-processing), it becomes particularly useful when we need to launch jobs say on remote HPCs. We might not receive a notice right away that our pre-processing failed because our data is not in bids format for example. Further it is a good practice on how to use containers. You can note however that 